In [10]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import base64
import sys

In [7]:
df=pd.read_csv("New_delhi.csv")

In [8]:
df

,Site,Type,Region,latitude,longitude,comments
0,Humayun's Tomb,Monument,Nizamuddin East,28.5933,77.2507,It was the first garden-tomb on the Indian sub...
1,Qutub Minar,Monument,South Delhi,28.5245,77.1855,Qutb Minar is a 73-metre tall tapering tower o...
2,Red Fort,Monument,Delhi,28.6562,77.2410,The Red Fort is a historic fort & it was the m...
3,India Gate,Monument,Delhi,28.6129,77.2295,The India Gate is a war memorial located astri...
4,Lotus Temple,Temple,Delhi,28.5535,77.2588,The Lotus Temple India is a Bahá'í House of Wo...
5,Akshardham Temple,Temple,New Delhi,28.6127,77.2773,The complex displays millennia of traditional ...
6,Connaught Place,Market,New Delhi,28.6304,77.2177,Connaught Place is a frenetic business and fin...
7,National Zoological Park,Zoo,Delhi,28.6067,77.2454,The National Zoological Park is a 176-acre zoo...
8,Lajpat Nagar,Market,South Delhi,28.5649,77.2403,Lajpat Nagar is centered on the sprawling Cent...


In [14]:
ksi_latitude = 28.6139; ksi_longitude = 77.2090
nd_map = folium.Map(location = [ksi_latitude,ksi_longitude], zoom_start = 6.4)
for lat, lng, borough, neighborhood, comments in zip(df['latitude'], df['longitude'], df['Region'], df['Site'],  df['comments']):
    label = '{}, {}, {}'.format(neighborhood, borough, comments)
    html = """ 
        <!doctype html>
        <html>
        <img src="2.jpg">
        </html>"""
    iframe = folium.IFrame(html=html,width=0.5,height=10.6)
    label = folium.Popup(label,iframe)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(nd_map)
nd_map

In [15]:
CLIENT_ID = 'OTPADGF3L0GOWJQ2EKPAZ0UW1QBWHXRYEXVARC0GCH4NRAP0'
CLIENT_SECRET = 'F2AUPFTLKVLWNC03TN5M3NTK1V4R0JBBNACPR2TM4ADKKVD5' 
VERSION = '20180805'

In [16]:
latitude_scar = 28.6139
longitude_scar = 77.2090

In [21]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2392d0fd16bb003946c019'},
 'response': {'headerLocation': 'N.D. Charge 4',
  'headerFullLocation': 'N.D. Charge 4, Delhi',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 28.62290000900001,
    'lng': 77.2192329929361},
   'sw': {'lat': 28.604899990999993, 'lng': 77.1987670070639}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56331e46498e862cbcde4360',
       'name': 'Hauz Khas Social',
       'location': {'lat': 28.6139391,
        'lng': 77.2090212,
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.6139391,
          'lng': 77.2090212}],
        'distance': 4,
        'cc': 'IN',
        'city': 'Delhi',
        'state': 'Delhi',
        'country': 'Indi

In [23]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Hauz Khas Social,Music Venue,28.613939,77.209021
1,Gurudwara Sri Rakabganj Sahibji,Spiritual Center,28.618296,77.205269
2,National Museum | राष्ट्रीय संग्रहालय (Nationa...,History Museum,28.611787,77.218493
3,Mughal Gardens | मुगल गार्डन (Mughal Gardens),Garden,28.617427,77.199727
4,Udyog Bhawan,Government Building,28.609466,77.211871
5,"Rashtrapati Bhawan, President's Estate Yeni De...",Museum,28.614350,77.199715
6,Mysore Cafe,Tea Room,28.608261,77.201475


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [27]:
nearby_venues


,name,categories,lat,lng
0,Hauz Khas Social,Music Venue,28.613939,77.209021
1,Gurudwara Sri Rakabganj Sahibji,Spiritual Center,28.618296,77.205269
2,National Museum | राष्ट्रीय संग्रहालय (Nationa...,History Museum,28.611787,77.218493
3,Mughal Gardens | मुगल गार्डन (Mughal Gardens),Garden,28.617427,77.199727
4,Udyog Bhawan,Government Building,28.609466,77.211871
5,"Rashtrapati Bhawan, President's Estate Yeni De...",Museum,28.614350,77.199715
6,Mysore Cafe,Tea Room,28.608261,77.201475


In [29]:
sites = getNearbyVenues(names=df['Site'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Humayun's Tomb
Qutub Minar
Red Fort
India Gate
Lotus Temple
Akshardham Temple
Connaught Place
National Zoological Park
Lajpat Nagar


In [30]:
sites.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Humayun's Tomb,28.5933,77.2507,Humayun’s Tomb | हुमायूँ का मकबरा (Humayun’s T...,28.593197,77.246047,Monument / Landmark
1,Humayun's Tomb,28.5933,77.2507,ISA KHAN’S GARDEN TOMB,28.592797,77.246147,Historic Site
2,Humayun's Tomb,28.5933,77.2507,Cafe Turtle,28.590371,77.252624,Café
3,Humayun's Tomb,28.5933,77.2507,Comesum - The Food Junction,28.589705,77.253212,Food Court
4,Humayun's Tomb,28.5933,77.2507,Bu-Halima's Garden,28.593111,77.246159,Garden


In [31]:
site_ven = pd.get_dummies(sites[['Venue Category']], prefix="", prefix_sep="")
site_ven['Neighborhood'] = sites['Neighborhood'] 
fixed_columns = [site_ven.columns[-1]] + list(site_ven.columns[:-1])
site_ven = site_ven[fixed_columns]
site_ven.head()

,Neighborhood,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beer Garden,Bistro,Brewery,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Donut Shop,Fast Food Restaurant,Flea Market,Food Court,Food Truck,Garden,Gastropub,Hindu Temple,Historic Site,Hotel,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Lounge,Market,Metro Station,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Nightclub,North Indian Restaurant,Park,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Restaurant,Salon / Barbershop,Sculpture Garden,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Sporting Goods Shop,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Zoo
0,Humayun's Tomb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Humayun's Tomb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Humayun's Tomb,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Humayun's Tomb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Humayun's Tomb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
scarb_grouped = site_ven.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beer Garden,Bistro,Brewery,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Donut Shop,Fast Food Restaurant,Flea Market,Food Court,Food Truck,Garden,Gastropub,Hindu Temple,Historic Site,Hotel,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Lounge,Market,Metro Station,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Nightclub,North Indian Restaurant,Park,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Restaurant,Salon / Barbershop,Sculpture Garden,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Sporting Goods Shop,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Zoo
0,Akshardham Temple,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00
1,Connaught Place,0.015625,0.0,0.015625,0.015625,0.000000,0.078125,0.015625,0.015625,0.015625,0.0,0.125000,0.0625,0.015625,0.078125,0.000000,0.015625,0.015625,0.078125,0.015625,0.0,0.000000,0.0,0.015625,0.0,0.015625,0.015625,0.078125,0.015625,0.03125,0.0,0.046875,0.000000,0.0,0.015625,0.000000,0.000000,0.0,0.015625,0.015625,0.000000,0.015625,0.000000,0.015625,0.015625,0.03125,0.015625,0.015625,0.000000,0.0,0.000000,0.000000,0.03125,0.000000,0.015625,0.0,0.000000,0.00
2,Humayun's Tomb,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.2,0.000000,0.0,0.200000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00
3,India Gate,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.00000,0.000000,0.000000,0.142857,0.0,0.142857,0.142857,0.00000,0.000000,0.000000,0.0,0.142857,0.00
4,Lajpat Nagar,0.000000,0.0,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.0,0.071429,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.071429,0.142857,0.071429,0.00000,0.0,0.000000,0.214286,0.0,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.071429,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.071429,0.000000,0.0,0.000000,0.00
5,Lotus Temple,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00
6,National Zoological Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.750000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000

In [33]:
from sklearn.cluster import KMeans
kclusters = 3
site_clust = scarb_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(site_clust)
kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 1, 2, 1, 1], dtype=int32)

In [34]:
import matplotlib.cm as cm
import matplotlib.colors as colors
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, ven, vc in zip(sites['Venue Latitude'], sites['Venue Longitude'], sites['Venue'], sites['Venue Category']):
    html = '<img src="image.png" style="width:400px;height:300px;">'
    label = folium.Popup( str(ven) +  str(vc),parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(nd_map)
nd_map